# Bonus projekt

#### Vojtěch Kváš (kvasvojt)



## 1. Generování dat

In [ ]:
import numpy as np
from statsmodels.tsa.arima_process import ArmaProcess

np.random.seed(506742)
ar = np.array([1, -0.5])
ma = np.array([1, 0.3])
process = ArmaProcess(ar, ma)
y = process.generate_sample(nsample=200)
y = np.cumsum(y)

In [ ]:
# pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_process import ArmaProcess

"""
df = pd.DataFrame({'x': y})
data = pd.Series(y, name="Syntetická časová řada")

"""

In [ ]:
from utils import *


tsplot(y)

## 2. Modelování

In [ ]:
#  pip install --upgrade numpy pmdarima

In [ ]:
# display(type(   data))
# display(type(   df))
display(type(y))

In [ ]:
#  pip install --upgrade pmdarima

In [ ]:
import pmdarima as pm

In [ ]:
import warnings

warnings.simplefilter("ignore")

print("Probiha automaticka volba radu modelu ARIMA...")
model_fit_auto = pm.auto_arima(
    y,
    start_p=1,
    start_q=1,
    test="adf",  # Pouziti ADF testu pro urceni d
    max_p=5,
    max_q=5,  # Maximalni rady p a q
    m=1,  # 1 pro ne-sezonni data
    d=None,  # Nechame auto_arima urcit d
    seasonal=False,  # Ne-sezonni model
    trace=True,
    #  scoring='aic',
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True,
    #  information_criterion='bic'
)

# Ziskani nejlepsiho radu (p, d, q)
best_order = model_fit_auto.order

print("\n" + "=" * 50)
print(f"1. ZVOLENÝ ŘÁD MODELU (p, d, q): {best_order}")
print("=" * 50 + "\n")

In [ ]:
import warnings

warnings.simplefilter("ignore")

print("Probiha automaticka volba radu modelu ARIMA...")
model_fit_auto = pm.auto_arima(
    y,
    start_p=1,
    start_q=1,
    test="adf",   
    max_p=5,
    max_q=5,  
    m=1,  
    d=None,  
    seasonal=False,  
    trace=True, 
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True,
    information_criterion='bic'
)

# Ziskani nejlepsiho radu (p, d, q)
best_order = model_fit_auto.order

print("\n" + "=" * 50)
print(f"1. ZVOLENÝ ŘÁD MODELU (p, d, q): {best_order}")
print("=" * 50 + "\n")

In [ ]:
 
auto_model = pm.auto_arima(
    y,
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    d=None,                       # automaticky testovat diference
    seasonal=False,               # nebo True s m=12 for měsíční data
    stepwise=False,               # heuristické prohledávání True/False
    information_criterion='bic',  # nebo 'bic'
    trace=True
)

print(auto_model.summary())

In [ ]:

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic

model = ARIMA(y, order=best_order, trend='n')
result = model.fit()
print(result.summary())

In [ ]:
result.plot_diagnostics(figsize=(9,9))
plt.show()

## 3. Validace

In [ ]:
from math import sqrt

aic_val = result.aic
bic_val = result.bic

 

In [ ]:
# Počáteční velikost trénovací sady (např. prvních 75 % dat)
initial_train_size = 1
# Horizont predikce (pro ROCV se typicky používá h=1)
forecast_horizon = 1 
# O kolik se okno posune v každé iteraci
step = 1

# Vytvoření seznamu pro ukládání chyb (skutečnost vs. predikce)
rolling_errors = []

# Velikost validace/testovací sady
n_test = len(y) - initial_train_size
print(f"Celková velikost dat: {len(y)}")
print(f"Počáteční trénovací sada: {initial_train_size} bodů")
print(f"Počet predikcí (Iterace): {n_test}\n")

# --- 3. Provedení ROCV (Pohyblivé okno) ---
# Iterujeme od bodu, kde končí počáteční trénovací sada, až do konce dat.
for i in range(initial_train_size, len(y), step):
    # Definice trénovací a testovací sady pro aktuální iteraci
    train_data = y[:i]
    # Skutečná hodnota, kterou se snažíme předpovědět
    actual_value = y[i:i + forecast_horizon] 
    
    # 1. Fitování modelu na aktuální trénovací sadě
 #   try:
    if True:
        print(f"Iterace {i - initial_train_size + 1}/{n_test}")
        print(f"Iterace {i - initial_train_size + 1}/{n_test}")
        model = ARIMA(train_data, order=best_order )
        results = model.fit( ) # disp=False potlačuje výpisy optimalizace
        
        # 2. Jednokroková predikce
        forecast = results.get_forecast(steps=forecast_horizon)
       #  display(forecast.summary_frame())
        display(forecast.predicted_mean[0])
        predicted_value = forecast.predicted_mean [0]
        
        # Uložení chyby predikce (skutečná hodnota - predikovaná hodnota)
        error = actual_value[0] - predicted_value
        rolling_errors.append(error)
        
        # print(f"Iterace {i - initial_train_size + 1}/{n_test}: Predikce {predicted_value:.2f}, Skutečnost {actual_value[0]:.2f}, Chyba: {error:.2f}")
"""
    except Exception as e:
        # Přeskočení iterace, pokud model selže (časté u extrémních dat nebo špatného řádu)
        print(f"Iterace {i - initial_train_size + 1} selhala: {e}")
        continue
"""
display(rolling_errors)

if rolling_errors:
    # RMSE je druhá odmocnina ze střední hodnoty čtverců chyb
    rolling_rmse = sqrt(np.mean(np.array(rolling_errors)**2))
    print("\n----------------------------------------------------")
    print(f"Rolling Origin RMSE (ARIMA{best_order}): {rolling_rmse:.3f}")
    print(f"Počet úspěšných predikcí: {len(rolling_errors)}")
    print("----------------------------------------------------")
else:
    print("\nNepodařilo se provést dostatečný počet predikcí pro výpočet RMSE.")

## 4. Predikce

## 4. Predikce